In [ ]:
import pandas as pd
import scipy.stats as stats
import scikit_posthocs as sp

In [ ]:
def plot_raincloud_with_stats_trial(parameters, params, n_trials=5, stimvals=[1e-3], pick_stim=1, conditions= ['wake', 'nmda', 'gaba', 'sleep'],  
                              colors = [ "steelblue", '#6d0a26', '#9b6369', '#c0b3b4'],
                              dx='stim', dy='PCI', dhue='cond', ort='v', sigma=0.2, local_folder=False):
    """
    df
    pick_stim: the stimulus for which you will compare
    colors should be at least as many as the conditions
    dx, dy, dhue, ort, sigma: parameters for the raincloud
    """
    if local_folder:
        print("Loading paper params:")
        params = [[5, 5.0, 60], [30, 3.75, 60], [30, 7.0, 60], [120, 5.0, 60]] # b_e, tau, nseeds
        conditions = ['wake', 'nmda', 'gaba', 'sleep'] #conditions that the params describe
        stimvals = [1e-5, 1e-4, 1e-3] #stimvals to load
        n_trials=5
        for i in range(len(conditions)):
            print(f"For {conditions[i]} : b_e={params[i][0]}, tau={params[i][1]}")
        print(f"Seeds = {params[0][2]}, n_trials={n_trials}, stimvals={stimvals}")
    PCI_all = create_PCI_all(parameters, params, n_trials=n_trials,stimvals = stimvals, local_folder=local_folder)

    print("Creating dataframe")
    df = create_dataset_for_raincloud(PCI_all, stimvals = stimvals, conditions= conditions)
#Check if there are more than one pick_stims:
    if type(pick_stim) is list and len(pick_stim)>1: 
        df_use = df[df['stim'].isin(pick_stim)]
        if len(np.unique(df_use['stim']))>1:
            plot_all_stimuli(df_use, sigma)
        else:
            print(f"Only stim = {np.unique(df_use['stim'])} exists for these parameters\nTry again with the correct value in pick_stim")
    else:
        if type(pick_stim) is list:
            pick_stim = pick_stim[0]
        df_use = df[df['stim']==pick_stim]
        # df_use = df
        if colors and len(colors)<=len(conditions):
            pal = sns.color_palette(colors)
        else:
            pal = sns.color_palette('tab20')
    print("Creating PCI_all")

    # df_use = df[df['stim']==pick_stim]
    return PCI_all, df_use

In [ ]:
params = [[5, 3.75, 2], [60, 5, 2]] # b_e, tau, nseeds
conditions = ['nmda', 'sleep'] #conditions that the params describe - used for the x tick labels
stimvals = [1e-3] #stimvals to load
n_trials=1 #how many trials were used for the PCI

pick_stim = [1] # stimulus strength to plot

#set this to True if you want to plot the already run stims, it will load params automatically
local_folder= True 


PCI_all, df = plot_raincloud_with_stats_trial(parameters, params, n_trials=n_trials, stimvals=stimvals, pick_stim=pick_stim, conditions= conditions, local_folder=local_folder)

In [ ]:
# Define the value and group columns as per the user's request
val_col = 'PCI'
group_col = 'cond'

# Rank the data
df['rank'] = df[val_col].rank()

# Perform the post-hoc Conover test
conover_results = sp.posthoc_conover(df, val_col=val_col, group_col=group_col, p_adjust='holm')

# Iterating over the pairs of groups in the dataset to calculate U and T-statistics
group_pairs = [(group1, group2) for i, group1 in enumerate(df[group_col].unique()) 
               for group2 in df[group_col].unique()[i+1:]]

In [ ]:
# Calculate and print the median for each group
median_values = df.groupby(group_col)[val_col].median()
print("Median values by group:")
print(median_values)

# Calculate and print the 25th and 75th percentiles for each group
quantiles = df.groupby(group_col)[val_col].quantile([0.25, 0.75]).unstack()
print("\n25th and 75th percentiles by group:")
print(quantiles)

In [ ]:
# Group the data
groups = df.groupby(group_col)[val_col].apply(list)

# Extract data for each group
group_data = [groups[group] for group in groups.index]

# Perform the Kruskal-Wallis H-test
kruskal_result = stats.kruskal(*group_data)

# Print the result
print(f"Krack-Wallis H-statistic: {kruskal_result.statistic}")
print(f"P-value: {kruskal_result.pvalue}")

In [ ]:
Post_hoc = sp.posthoc_conover(df, val_col='PCI', group_col='cond', p_adjust='holm')
Post_hoc

In [ ]:
# Calculating the T-statistic for each pair
t_stats = {}
for group1, group2 in group_pairs:
    groupA = df[df[group_col] == group1]['rank']
    groupB = df[df[group_col] == group2]['rank']
    
    # Mann-Whitney U test between two groups
    u_stat, p_value = stats.mannwhitneyu(groupA, groupB, alternative='two-sided')
    
    # Convert U-statistic to T-statistic approximation
    n1 = len(groupA)
    n2 = len(groupB)
    
    print(n1, n2)
    # Calculate the T-statistic (based on U approximation)
    t_stat = (u_stat - n1 * n2 / 2) / (n1 * n2 * (n1 + n2 + 1) / 12)**0.5
    
    # Store the result
    t_stats[(group1, group2)] = t_stat

# Output the T-statistics for each group pair
t_stats

In [ ]:

# Sample size for each group
sample_size = 60

# Function to calculate effect size r
def calculate_effect_size_r(t_stat, sample_size):
    total_n = 2 * sample_size  # Since each comparison involves two groups
    r = t_stat / np.sqrt(total_n)
    return r

# Dictionary to store the effect sizes
effect_sizes = {}

# Calculate the effect size r for each pairwise comparison
for pair, t_stat in t_stats.items():
    r = calculate_effect_size_r(t_stat, sample_size)
    effect_sizes[pair] = r

# Output the effect sizes
effect_sizes